#### Association Object

The _association object pattern_ is a _variant on many-to-many_: it's used when your _association table_ __contains additional columns beyond those which are foreign keys__ to the left and right tables. Instead of using the `relationship.secondary` argument, you _map a new class directly_ to the association table. The _left side_ of the relationship references the _association object via one-to-many_, and the _association class references the right side via many-to-one_. Below we illustrate an association table mapped to the Association class which includes a column called extra_data, which is a string value that is stored along with each association between Parent and Child.

In [3]:
from sqlalchemy import Column, ForeignKey, Integer, String
from sqlalchemy.orm import declarative_base, relationship

In [2]:
Base = declarative_base()

In [4]:
class Association(Base):
    __tablename__ = "association_table"
    left_id = Column(ForeignKey("left_table.id"), primary_key=True)
    right_id = Column(ForeignKey("right_table.id"), primary_key=True)
    extra_data = Column(String(50))
    child = relationship("Child")

In [5]:
class Parent(Base):
    __tablename__ = "left_table"
    id = Column(Integer, primary_key=True)
    children = relationship("Association")

In [6]:
class Child(Base):
    __tablename__ = "right_table"
    id = Column(Integer, primary_key=True)

As always, the _bidirectional_ version makes use of `relationship.back_populates` or `relationship.backref`.

In [7]:
class AssociationModel(Base):
    __tablename__ = "association"
    left_id = Column(ForeignKey("left.id"), primary_key=True)
    right_id = Column(ForeignKey("right.id"), primary_key=True)
    extra_data = Column(String(50))
    child = relationship("ChildModel", back_populates="parents")
    parent = relationship("ParentModel", back_populates="children")

In [8]:
class ParentModel(Base):
    __tablename__ = "left"
    id = Column(Integer, primary_key=True)
    children = relationship("AssociationModel", back_populates="parent")

In [9]:
class ChildModel(Base):
    __tablename__ = "right"
    id = Column(Integer, primary_key=True)
    parents = relationship("AssociationModel", back_populates="child")

Working with the _association pattern_ in its _direct form_ __requires__ that __child objects are associated with an association instance before being appended to the parent__; similarly, _access from parent to child goes through the association object_:

```
# create parent, append a child via association
p = ParentModel()
a = AssociationModel(extra_data="some data")
a.child = ChildModel()
p.children.append(a)

# iterate through child objects via association, including association attributes
for assoc in p.children:
    print(assoc.extra_data)
    print(assoc.child)
```

To enhance the _association object pattern_ such that __direct access__ to the _Association_ object is __optional__, SQLAlchemy provides the __Association Proxy extension__. This extension allows the _configuration of attributes_ which will __access two "hops" with a single access__, one "hop" to the _associated object_, and a second to a _target attribute_.